In [ ]:
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
age_begin = 13
age_end = 100
risk_groups = ['HM', 'HF', 'MSM']
num_risk = 3

all_jur = 10
age_groups = 88
number_of_risk_groups = 3
number_of_compartments = 22
dt = 1/12

num_age = 88
num_comp = number_of_compartments-2

prep_efficiency = 0.99

unaware_index = (1,5,9,13,17)
aware_no_care_index = (2,6,10,14,18)
ART_VLS_index = (3,4,7,8,11,12,15,16,19,20)
VLS_index = (4,8,12,16,20)


pop_growth_rate = 0

gamma = np.array([[0.5,0.5,0.5,0.5,1],
                  [0.5,0.5,0.5,0.5,1],
                  [0.5,0.5,0.5,0.5,1]])

scaling_factor_dropout = np.array([[1,1,1,1,1,1,1,1,0,0],
                                   [1,1,1,1,1,1,1,1,0,0],
                                   [1,1,1,1,1,1,1,1,0,0]])

all_jurisdictions = [6001,6037,6059,6065,6067,6071,6073,6]

num_jur = len(all_jurisdictions)

cluster1 = [6001]
cluster2 = [6037]
cluster3 = [6059]
cluster4 = [6065]
cluster5 = [6067]
cluster6 = [6071]
cluster7 = [6073]
cluster8 = [6]

cluster1_index = np.array([0])
cluster2_index = np.array([1])
cluster3_index = np.array([2])
cluster4_index = np.array([3])
cluster5_index = np.array([4])
cluster6_index = np.array([5])
cluster7_index = np.array([6])
cluster8_index = np.array([7])

In [ ]:
theta = 0.3
tau = 0.3
mu = 0.002
alpha = 0.8

c_r_n = 22.13
c_c_n = 10.36
n_n = 0.45
c_add = 52.66

c_r_p = 78.8
c_c_p = 58.91
n_r_p = 10.86
n_c_p = 5.88
c_cnf = 160.07

O_v0 = 16.59
W = 0.2 #[0.1,0.2,0.3]
# del_x = #outreached people

m_cl = 1000
m_nc = 1000
f_c = 56379
f_nc = 64851

m_o = 1000
f_o = 50000

R_art_0 = 235 #[117,235,300]
Y = 0.2#[0.1,0.2,0.3]
m_r = 500
f_r = 22708 #[17977,22708,29330]

# p_art_0 = np.array([0.51041812, 0.5950631 , 0.57486548]) #need for all jur

In [ ]:
x_t_a_0 = np.array([[2.33180375e+03, 3.63425551e+03, 3.04112649e+02],
       [3.17936973e+04, 2.75670559e+04, 1.47418143e+03],
       [1.36887484e+04, 1.30437095e+04, 5.32839478e+02],
       [4.90237946e+03, 9.09907366e+03, 5.72640202e+02],
       [1.47264430e+03, 3.48346071e+03, 3.16193026e+02],
       [3.62561698e+03, 7.52180368e+03, 1.63632454e+02],
       [5.32412773e+03, 1.15369335e+04, 4.73913288e+02],
       [8.82399246e+02, 5.86348541e+02, 2.57480493e+02]])

delta_1 = np.array([[0.0180713 , 0.03533054, 0.0314886 ],
       [0.05123306, 0.059546  , 0.03496893],
       [0.04669086, 0.05902862, 0.0339207 ],
       [0.02449952, 0.0553693 , 0.03036947],
       [0.01170278, 0.03310768, 0.03407533],
       [0.01437995, 0.03664002, 0.01971395],
       [0.01977946, 0.05301006, 0.02548554],
       [0.038004  , 0.031486  , 0.06326625]])

p_unaware_0 = np.array([[0.17989 , 0.12448 , 0.167303],
       [0.132271, 0.090542, 0.123574],
       [0.196625, 0.137524, 0.184114],
       [0.196059, 0.138523, 0.185914],
       [0.208705, 0.142397, 0.188764],
       [0.282237, 0.20057 , 0.260866],
       [0.186459, 0.130231, 0.174968],
       [0.078656, 0.05356 , 0.074505]])

p_art_0 = np.array([[0.565534, 0.647454, 0.627884],
       [0.570325, 0.645897, 0.636072],
       [0.536505, 0.620825, 0.599717],
       [0.496017, 0.579814, 0.561497],
       [0.646201, 0.72944 , 0.697714],
       [0.543873, 0.639447, 0.6001  ],
       [0.53376 , 0.616783, 0.597922],
       [0.547729, 0.617069, 0.619356]])

In [ ]:
c_l = 54000

cd4_gt_350_index = (1,2,3,4,5,6,7,8,9,10,11,12)
cd4_200_350_index = (13,14,15,16)
cd_lt_200_index = (17,18,19,20)

QALY_val_gt_350 = 0.935
QALY_val_250_350 = 0.818
QALY_val_lt_200 = 0.702

In [ ]:
def shi_value(I, N):
    return (0.0153/0.0057)*(I/N)

In [ ]:
def cost(data_t_1, data_t, unaware_prop, aware_art_vls_prop, diagnosis_rate_risk, dropout_rate_risk, prep_rate):

    cost_risk = np.zeros((num_jur, number_of_risk_groups))

    for risk in range(len(risk_groups)):
        
        total_pop_t_risk = np.apply_over_axes(np.sum, data_t[:,risk,:,0:21],[1,2]).reshape(num_jur,)
        I_t_1 = np.apply_over_axes(np.sum, data_t_1[:,risk,:,1:21],[1,2]).reshape(num_jur,)
        p_t_1_unaware = unaware_prop[:,risk]
        p_t_1_art_vls = aware_art_vls_prop[:,risk]
        delta_t = diagnosis_rate_risk[:,risk]
        dropout_t = dropout_rate_risk[:,risk]
        r_t_a_risk = delta_t*I_t_1*p_t_1_unaware 
        shi = shi_value(I_t_1, total_pop_t_risk)
        x_t_a_risk = (r_t_a_risk - mu*I_t_1*p_t_1_unaware) / (theta*shi)
        n_t_a_risk = mu*(total_pop_t_risk - I_t_1*p_t_1_unaware - x_t_a_risk) + x_t_a_risk*theta*(1-shi) 
        X_v_risk = tau*c_r_n + (1-tau)*c_c_n + n_n + (1-alpha)*c_add
        Y_v_risk = tau*(c_r_p + n_r_p) + (1-tau)*(c_c_p + n_c_p) + c_cnf + (1-alpha)*c_add
        
        del_x = (x_t_a_risk - x_t_a_0[:,risk])/total_pop_t_risk
        
        O_v = O_v0*np.exp(del_x*W)
        
        X_f_cl_a = ((r_t_a_risk + n_t_a_risk)*alpha / m_cl)*f_c
        X_f_ncl_a = ((r_t_a_risk + n_t_a_risk)*(1 - alpha) / m_nc)*f_nc
        X_f_o_a = (x_t_a_risk / m_o)*f_o

        cost_of_testing = r_t_a_risk*Y_v_risk + x_t_a_risk* O_v + n_t_a_risk*X_v_risk + X_f_cl_a + X_f_ncl_a + X_f_o_a

        d_t_a_risk = (1 - dropout_t)*I_t_1*p_t_1_art_vls
        del_p_art = p_t_1_art_vls - p_art_0[:,risk]
        R_v_risk = R_art_0*np.exp(del_p_art*Y)
        E_f_a = (d_t_a_risk / m_r)*f_r

        cost_retention_in_care = d_t_a_risk*R_v_risk + E_f_a

        if risk == 2:

            prep_adherence_per_person_per_year = 1431
            prep_medication_per_person_per_year = 12599

            prep_cost = np.sum(data_t_1[:,risk,:,0], axis=1)*prep_rate[:,risk]* \
                            (prep_adherence_per_person_per_year + prep_medication_per_person_per_year)

        else:
            prep_cost = 0

        cost_risk[:,risk] = cost_of_testing + cost_retention_in_care + prep_cost
        
    return dt*cost_risk

In [ ]:
def benefit(data_array):
    L_t_risk = np.zeros((num_jur, number_of_risk_groups))

    for risk in range(len(risk_groups)):

        num_uninfected = np.sum(data_array[:,risk,:,0], axis=1)
        num_over_350 = np.apply_over_axes(np.sum, data_array[:,risk,:,cd4_gt_350_index], [0,2]).reshape(num_jur,)
        num_250_350 = np.apply_over_axes(np.sum, data_array[:,risk,:,cd4_200_350_index], [0,2]).reshape(num_jur,)
        num_below_250 = np.apply_over_axes(np.sum, data_array[:,risk,:,cd_lt_200_index], [0,2]).reshape(num_jur,)

        benefit_risk = 1*num_uninfected + QALY_val_gt_350*num_over_350 + QALY_val_250_350*num_250_350 + QALY_val_lt_200*num_below_250

        L_t_risk[:,risk] = benefit_risk
        
    return c_l*dt*L_t_risk